### Basic working of Google Palm LLM in LangChain

In [2]:


# ✅ NEW IMPORT for Gemini
import google.generativeai as genai

# ✅ Other imports as you had
from langchain.chains import RetrievalQA
from langchain.document_loaders.csv_loader import CSVLoader
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import FAISS
from langchain.prompts import PromptTemplate
import os

In [3]:

# ✅ Step 1: Setup Gemini API
api_key = os.environ['GOOGLE_API_KEY']  # Replace with your API key
genai.configure(api_key=api_key)

In [7]:
for m in genai.list_models():
    print(m.name, " | ", m.supported_generation_methods)


models/chat-bison-001  |  ['generateMessage', 'countMessageTokens']
models/text-bison-001  |  ['generateText', 'countTextTokens', 'createTunedTextModel']
models/embedding-gecko-001  |  ['embedText', 'countTextTokens']
models/gemini-1.0-pro-vision-latest  |  ['generateContent', 'countTokens']
models/gemini-pro-vision  |  ['generateContent', 'countTokens']
models/gemini-1.5-pro-latest  |  ['generateContent', 'countTokens']
models/gemini-1.5-pro-001  |  ['generateContent', 'countTokens', 'createCachedContent']
models/gemini-1.5-pro-002  |  ['generateContent', 'countTokens', 'createCachedContent']
models/gemini-1.5-pro  |  ['generateContent', 'countTokens']
models/gemini-1.5-flash-latest  |  ['generateContent', 'countTokens']
models/gemini-1.5-flash-001  |  ['generateContent', 'countTokens', 'createCachedContent']
models/gemini-1.5-flash-001-tuning  |  ['generateContent', 'countTokens', 'createTunedModel']
models/gemini-1.5-flash  |  ['generateContent', 'countTokens']
models/gemini-1.5-fla

In [4]:
from langchain_core.language_models import LLM
from typing import Optional, List
from langchain_core.outputs import Generation, LLMResult
import google.generativeai as genai
from pydantic import PrivateAttr

class GeminiLLM(LLM):
    model_name: str = "models/gemini-pro"
    temperature: float = 0.1
    api_key: Optional[str] = None

    # 👉 Private attribute for internal model
    _model: any = PrivateAttr()

    def __init__(self, model_name: Optional[str] = None, temperature: float = 0.1, api_key: Optional[str] = None):
        super().__init__()
        if api_key:
            genai.configure(api_key=api_key)
        self.model_name = model_name or self.model_name
        self.temperature = temperature
        self.api_key = api_key
        self._model = genai.GenerativeModel(self.model_name)

    @property
    def _llm_type(self) -> str:
        return "gemini"

    def _call(self, prompt: str, stop: Optional[List[str]] = None) -> str:
        response = self._model.generate_content(
            prompt,
            generation_config={"temperature": self.temperature}
        )
        return response.text

    def _generate(self, prompts: List[str], stop: Optional[List[str]] = None) -> LLMResult:
        generations = []
        for prompt in prompts:
            text = self._call(prompt, stop)
            generations.append([Generation(text=text)])
        return LLMResult(generations=generations)


In [5]:
# ✅ Step 3: Initialize Gemini LLM
llm = GeminiLLM(
    model_name="models/gemini-1.5-pro-latest",
    temperature=0.1,
    
)

# ✅ Step 4: Quick test - Generate poem
poem = llm("Write a 4 line poem of my love for biryani")
print(poem)

C:\Users\omkum\AppData\Local\Temp\ipykernel_18596\3830560266.py:9: LangChainDeprecationWarning: The method `BaseLLM.__call__` was deprecated in langchain-core 0.1.7 and will be removed in 1.0. Use :meth:`~invoke` instead.
  poem = llm("Write a 4 line poem of my love for biryani")


Oh, biryani, fragrant, spiced delight,
Each layered grain a flavorful bite,
From saffron hues to meat so tender,
My love for you will never surrender. 



In [7]:
# ✅ Step 5: Load your data
loader = CSVLoader(file_path='college_questions_answers.csv', source_column="Question")
data = loader.load()

In [8]:
# ✅ Step 6: Create Embedding model
model_name = "sentence-transformers/all-MiniLM-L6-v2"
embedding_model = HuggingFaceEmbeddings(model_name=model_name)

C:\Users\omkum\AppData\Local\Temp\ipykernel_18596\3787025548.py:3: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embedding_model = HuggingFaceEmbeddings(model_name=model_name)


In [9]:
query = "What courses or majors are offered?"
embedding = embedding_model.embed_query(query)
print(embedding[:5])

[0.0812506228685379, -0.027420220896601677, 0.03651316836476326, 0.059323567897081375, -0.06641454994678497]


In [ ]:
''''from langchain.chains import RetrievalQA


from langchain.embeddings import GooglePalmEmbeddings
from langchain.llms import GooglePalm'''

### Now let's load data from Codebasics FAQ csv file

In [ ]:
''''from langchain.document_loaders.csv_loader import CSVLoader

loader = CSVLoader(file_path='College_Admission_Questions_and_Answers.csv', source_column="Prompt")

# Store the loaded data in the 'data' variable
data = loader.load()'''

### Hugging Face Embeddings

In [ ]:
'''from langchain.embeddings import HuggingFaceEmbeddings

model_name = "sentence-transformers/all-MiniLM-L6-v2"
embedding_model = HuggingFaceEmbeddings(model_name=model_name)

query = "What courses or majors are offered?"
embedding = embedding_model.embed_query(query)
print(embedding[:5])'''


C:\Users\omkum\AppData\Local\Temp\ipykernel_6832\2071888529.py:4: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embedding_model = HuggingFaceEmbeddings(model_name=model_name)


[0.0812506228685379, -0.027420220896601677, 0.03651316836476326, 0.059323567897081375, -0.06641454994678497]


In [10]:
len(embedding)

384

In [11]:
embedding[:5]

[0.0812506228685379,
 -0.027420220896601677,
 0.03651316836476326,
 0.059323567897081375,
 -0.06641454994678497]

As you can see above, embedding for a sentance "What is your refund policy" is a list of size 768. Looking at the numbers in this list, doesn't give any intuitive understanding of what it is but just assume that these numbers are capturing the meaning of "What is your refund policy". If you are curious to know about embeddings, go to youtube and search "codebasics word embeddings" and you will find bunch of videos with simple, intuitive explanations

### Vector store using FAISS

In [12]:


from langchain.vectorstores import FAISS

# Create a FAISS instance for vector database from 'data'
vectordb = FAISS.from_documents(documents=data,
                                 embedding=embedding_model)

# Create a retriever for querying the vector database
retriever = vectordb.as_retriever(score_threshold = 0.7)

In [13]:
rdocs = retriever.get_relevant_documents("Are internships or practical experiences required?")
rdocs

C:\Users\omkum\AppData\Local\Temp\ipykernel_18596\786521673.py:1: LangChainDeprecationWarning: The method `BaseRetriever.get_relevant_documents` was deprecated in langchain-core 0.1.46 and will be removed in 1.0. Use :meth:`~invoke` instead.
  rdocs = retriever.get_relevant_documents("Are internships or practical experiences required?")


[Document(id='7217c10a-9f26-486a-bad8-8048e72024a1', metadata={'source': 'Are internships mandatory? Does the college help in internships?', 'row': 25}, page_content='Question: Are internships mandatory? Does the college help in internships?\nAnswer: Yes. Internships are strongly encouraged, and the college assists students in securing internship opportunities.'),
 Document(id='c4bc5b24-4363-4934-80f2-34c4c9e4a147', metadata={'source': 'Are there opportunities for research or internships?', 'row': 5}, page_content='Question: Are there opportunities for research or internships?\nAnswer: Yes. Students are encouraged to participate in research activities and internships through dedicated support systems.'),
 Document(id='83c68357-0378-42ac-80e5-fa08e96a4810', metadata={'source': 'Is there a dedicated placement cell?', 'row': 26}, page_content='Question: Is there a dedicated placement cell?\nAnswer: Yes. A specialized placement cell provides training, career counseling, and placement assis

As you can see above, the retriever that was created using FAISS and hugging face embedding is now capable of pulling relavant documents from our original CSV file knowledge store. This is very powerful and it will help us further in our project

##### Embeddings can be created using GooglePalm too. Also for vector database you can use chromadb as well as shown below. During our experimentation, we found hugging face embeddings and FAISS to be more appropriate for our use case

In [14]:
# google_palm_embeddings = GooglePalmEmbeddings(google_api_key=api_key)

# from langchain.vectorstores import Chroma
# vectordb = Chroma.from_documents(data,
#                            embedding=google_palm_embeddings,
#                            persist_directory='./chromadb')
# vectordb.persist()

### Create RetrievalQA chain along with prompt template 🚀

In [14]:
# ✅ Step 11: Prompt Template
prompt_template = """Given the following context and a question, generate an answer based on this context only.
In the answer try to provide as much text as possible from "Answer" section in the source document context without making much changes.
If the answer is not found in the context, kindly state "I don't know." Don't try to make up an answer.

CONTEXT: {context}

QUESTION: {question}"""

PROMPT = PromptTemplate(
    template=prompt_template, input_variables=["context", "question"]
)
chain_type_kwargs = {"prompt": PROMPT}

In [15]:
# ✅ Step 12: Create RetrievalQA Chain
chain = RetrievalQA.from_chain_type(
    llm=llm,
    chain_type="stuff",
    retriever=retriever,
    input_key="query",
    return_source_documents=True,
    chain_type_kwargs=chain_type_kwargs
)

### We are all set 👍🏼 Let's ask some questions now

In [16]:
chain('How is the curriculum?')

C:\Users\omkum\AppData\Local\Temp\ipykernel_18596\2683687648.py:1: LangChainDeprecationWarning: The method `Chain.__call__` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use :meth:`~invoke` instead.
  chain('How is the curriculum?')


{'query': 'How is the curriculum?',
 'result': 'Yes. The curriculum is designed to be modern and industry-aligned, covering the latest trends and technologies.\n',
 'source_documents': [Document(id='65d4afaa-4c2c-41d9-8234-695e89957e54', metadata={'source': 'Is the curriculum updated and industry-relevant?', 'row': 4}, page_content='Question: Is the curriculum updated and industry-relevant?\nAnswer: Yes. The curriculum is designed to be modern and industry-aligned, covering the latest trends and technologies.'),
  Document(id='936e398d-51ee-42d3-a5e7-1143b09b279f', metadata={'source': 'How is the faculty? Are they experienced and qualified?', 'row': 2}, page_content='Question: How is the faculty? Are they experienced and qualified?\nAnswer: Yes. The faculty is highly qualified and experienced, focused on providing quality education and mentorship.'),
  Document(id='7dbd5f57-7315-4c8a-852d-43a8570b848b', metadata={'source': 'How are the library, labs, and study resources?', 'row': 20}, 

**As you can see above, the answer of question comes from two different FAQs within our csv file and it is able to pull those questions and merge them nicely**

In [18]:
chain("Are there scholarships or financial aid options?")

{'query': 'Are there scholarships or financial aid options?',
 'result': 'Yes. Scholarships are available for academically outstanding students and those from economically weaker backgrounds.\n',
 'source_documents': [Document(id='9ac1ac44-dd4c-427d-8bf0-1a6dd9018829', metadata={'source': 'Are there scholarships or financial aid options?', 'row': 16}, page_content='Question: Are there scholarships or financial aid options?\nAnswer: Yes. Scholarships are available for academically outstanding students and those from economically weaker backgrounds.'),
  Document(id='06872282-d34a-4ab7-9feb-a80d6dcc1374', metadata={'source': 'What are the eligibility criteria for scholarships?', 'row': 17}, page_content='Question: What are the eligibility criteria for scholarships?\nAnswer: Yes. Scholarships are typically awarded based on academic performance and financial need.'),
  Document(id='c4bc5b24-4363-4934-80f2-34c4c9e4a147', metadata={'source': 'Are there opportunities for research or internshi

In [19]:
chain("How good are the science/lab facilities and Are computer labs available 24/7?")

{'query': 'How good are the science/lab facilities and Are computer labs available 24/7?',
 'result': "Yes. The institution provides a well-equipped library, modern labs, and comprehensive academic resources.  I don't know if computer labs are available 24/7.\n",
 'source_documents': [Document(id='7dbd5f57-7315-4c8a-852d-43a8570b848b', metadata={'source': 'How are the library, labs, and study resources?', 'row': 20}, page_content='Question: How are the library, labs, and study resources?\nAnswer: Yes. The institution provides a well-equipped library, modern labs, and comprehensive academic resources.'),
  Document(id='7ceb1c96-e2cb-42cb-87a1-48cc8ca022de', metadata={'source': 'Is the campus Wi-Fi-enabled and tech-friendly?', 'row': 19}, page_content='Question: Is the campus Wi-Fi-enabled and tech-friendly?\nAnswer: Yes. The campus is Wi-Fi-enabled and supports a tech-friendly learning environment.'),
  Document(id='df21620a-f279-4d9d-88d8-56cb2724cbe8', metadata={'source': 'What are th

In [20]:
chain("What courses or programs are offered?")

{'query': 'What courses or programs are offered?',
 'result': 'Yes. A diverse range of undergraduate and postgraduate programs is offered across fields like Computer Science, Commerce, Management, and IT.\n',
 'source_documents': [Document(id='d52a6d63-6e51-45e5-b343-852bd732ee1a', metadata={'source': 'What courses or programs are offered?', 'row': 0}, page_content='Question: What courses or programs are offered?\nAnswer: Yes. A diverse range of undergraduate and postgraduate programs is offered across fields like Computer Science, Commerce, Management, and IT.'),
  Document(id='c41b921c-19f2-42b3-8266-f4bba90d39ce', metadata={'source': 'How flexible are the course choices (electives, minors)?', 'row': 6}, page_content='Question: How flexible are the course choices (electives, minors)?\nAnswer: Yes. Multiple electives and value-added courses are available, offering students flexibility in their learning paths.'),
  Document(id='c8cc50cd-58dc-4a96-8edb-0d4f7d93e005', metadata={'source':

In [20]:
chain("should I learn power bi or tableau?")

{'query': 'should I learn power bi or tableau?',
 'result': 'This is a contextual question. If you are talking about a pure visualization tool Tableau is slightly better. Data connectors, modeling and transformation features are available in both. However, factually speaking Power BI is cheaper and offers tighter integration with the Microsoft environment. Since most companies use excel & Microsoft tools they start with Power BI or move towards Power BI for seamless integration with other Microsoft tools (called as Power platform). This makes the job openings grow at a much higher rate on Power BI and Power Platform. Also, Power BI has been leading the Gartner’s magic quadrant in BI for the last few years as the industry leader.',
 'source_documents': [Document(page_content='prompt: Power BI or Tableau which one is better?\nresponse: This is a contextual question. If you are talking about a pure visualization tool Tableau is slightly better. Data connectors, modeling and transformation

In [21]:
chain("I've a MAC computer. Can I use powerbi on it?")

{'query': "I've a MAC computer. Can I use powerbi on it?",
 'result': 'response: Hi\n\nPower BI desktop works only in Windows OS. Please look into the system requirements section on this page. However, you can use a virtual machine to install and work with Power BI in other Operating systems.',
 'source_documents': [Document(page_content='prompt: How can I use PowerBI on my Mac system?\nresponse: Hi\n\nYou can use VirtualBox to create a virtual machine and install Windows on it. This will allow you to run Power BI and Excel on your Mac.\n\nIf you\'re not familiar with setting up a virtual machine, there are many resources available on YouTube that can guide you through the process. Simply search for "installing virtual machines" and you\'ll find plenty of helpful videos.\n\nBest of luck with your studies!', metadata={'source': 'How can I use PowerBI on my Mac system?', 'row': 44}),
  Document(page_content='prompt: Does Power BI work in Mac OS/Ubuntu?\nresponse: Power BI desktop works o

In [ ]:
chain("I don't see power pivot. how can I enable it?")

In [ ]:
chain("What is the price of your machine learning course?")